In [15]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import matplotlib.pyplot as plt
from hyperion.model import ModelOutput
from astropy.cosmology import Planck13
from astropy import units as u
from astropy import constants
from hyperion.util.constants import pc
import h5py
import os
import sys
import ytree
import yt
np.set_printoptions(threshold=sys.maxsize)
yt.enable_plugins()

yt : [INFO     ] 2025-04-08 13:56:29,590 Loading plugins from /storage/home/hcoda1/7/shardin31/.config/yt/my_plugins.py
yt : [WARNING  ] 2025-04-08 13:56:29,594 Field ('gas', 'healpix_pix_3') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-04-08 13:56:29,595 Field ('gas', 'metallicity3') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-04-08 13:56:29,595 Field ('gas', 'total_metallicity') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-04-08 13:56:29,596 Field ('gas', 'electron_fraction') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-04-08 13:56:29,598 Field ('gas', 'J21_LW') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-04-08 13:56:29,599 Field ('gas', 'J_LW') already exists. To override use `force_override=True`.
yt : [WARNING  ] 2025-04-08 13:56:29,599 Field ('gas', 'J_Lyman') already exists. To override use `force_override=True`.
yt : [

In [16]:
#apparent magnitude
def app_mag(z, run):
    
    filter_directory = '/storage/home/hcoda1/7/shardin31/p-jw254-0/Research/Powderday/powderday/filters/mean_throughputs_downsized/'
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)

    m = ModelOutput(run)
    wav,flux = m.get_sed(inclination='all',aperture=-1)

    wav  = np.asarray(wav)*u.micron #wav is in micron
    wav *= (1.+z)

    flux = np.asarray(flux)*u.erg/u.s
    dl = Planck13.luminosity_distance(z)
    dl = dl.to(u.cm)
    
    flux /= (4.*3.14*dl**2.)

    nu = constants.c.cgs/(wav.to(u.cm))
    nu = nu.to(u.Hz)

    flux /= nu
    flux = flux.to(u.mJy)


    micron  = []
    throughput = []
    x = 0
    
    app_mag_array = {}
    
#     for filters in os.listdir(filter_directory):
#         app_mag_array[filters] = np.zeros(flux.shape[0])
    
    filter_arr = ['F070W', 'F090W', 'F115W', 'F150W', 'F200W', 'F277W', 'F356W', 'F444W']
    
    for filters in filter_arr:
        app_mag_array[filters] = np.zeros(flux.shape[0])
    
    for i in range(flux.shape[0]):
#         for filters in os.listdir(filter_directory):
#             f = open(filter_directory + filters, "r")
#             for line in f:
#                 row = line.split()
#                 micron.append(row[0])
#                 throughput.append(row[1])
        for filters in filter_arr:
            with open('/storage/home/hcoda1/7/shardin31/p-jw254-0/Research/Powderday/powderday/filters/mean_throughputs_downsized/'+str(filters)+'_downsized.txt', 'r') as f:
                for line in f:
                    micron.append(float(line.split()[0]))
                    throughput.append(float(line.split()[1]))
            micron = np.asarray(micron)
            throughput = np.asarray(throughput)
            micron = micron.astype(np.float64) * (1+z)
            throughput = throughput.astype(np.float64)
            micron *= u.micron
            min_lam = max(wav.min(), micron.min())
            max_lam = min(wav.max(), micron.max())
            flipped_wav = wav[::-1]
            min_idx = np.searchsorted(flipped_wav, min_lam, side="left")
            max_idx = np.searchsorted(flipped_wav, max_lam, side="left")
            lam_clip = flipped_wav[min_idx : max_idx]
            throughput_interp = np.interp(lam_clip, micron, throughput)
            flipped_flux = flux[i][::-1]
            sed_clip = flipped_flux[min_idx : max_idx]
            #print("sed clip", sed_clip)
            sed_final = np.trapz(sed_clip * throughput_interp)
            #print("sed final", sed_final)
            zero_point = 3.63078e6 * np.trapz(throughput_interp) * u.mJy
            apparent_magnitude = -2.5 * np.log10(sed_final / zero_point)
            #print("app mag", apparent_magnitude)
            app_mag_array[filters][i] = apparent_magnitude
            del micron
            del throughput
            micron = []
            throughput = []
            x += 1
        x = 0
    
    return(app_mag_array)


In [17]:
a = ytree.load("/storage/home/hcoda1/0/jw254/data/SG64-2020/rockstar_halos-jhw/trees/tree_0_0_0.dat")
run = '/storage/home/hcoda1/7/shardin31/p-jw254-0/Research/summer2023/pd_test/run_10000/halo_1/example.0001.rtout.sed'


filename = '/storage/home/hcoda1/0/jw254/data/SG64-2020/DD0125/output_0125'
ds = yt.load(filename)
z = ds.current_redshift

app_mag_final_arr = app_mag(z, run)
abs_mag_final_arr = abs_mag(z, run)

filters = ['F070W', 'F090W', 'F115W', 'F150W', 'F200W', 'F277W', 'F356W', 'F444W']

color_bands_over_all_inclination_angles = {}

for i in range(0, len(filters), 1):
    if i < (len(filters) - 1):
        color_bands_over_all_inclination_angles[filters[i]+"-"+filters[i+1]] = (app_mag_final_arr[filters[i]] - app_mag_final_arr[filters[i+1]])     
    
print(color_bands_over_all_inclination_angles)

# print("apparent magnitude")
# print(app_mag_final_arr)

# print("\n\n")
# print("absolute magnitude")
# print(abs_mag_final_arr)
#print(abs_mag_final_arr)

yt : [INFO     ] 2025-04-08 13:56:42,781 Parameters: current_time              = 75.89987395114
yt : [INFO     ] 2025-04-08 13:56:42,783 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2025-04-08 13:56:42,784 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-04-08 13:56:42,785 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-04-08 13:56:42,786 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2025-04-08 13:56:42,786 Parameters: current_redshift          = 11.181356517874
yt : [INFO     ] 2025-04-08 13:56:42,787 Parameters: omega_lambda              = 0.6889
yt : [INFO     ] 2025-04-08 13:56:42,787 Parameters: omega_matter              = 0.3111
yt : [INFO     ] 2025-04-08 13:56:42,788 Parameters: omega_radiation           = 0
yt : [INFO     ] 2025-04-08 13:56:42,788 Parameters: hubble_constant           = 0.6766
/tmp/ipykernel_2625417/3332539215.py:23: RuntimeWarning: More than 20 figures have been opened. Figur

{'F070W-F090W': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'F090W-F115W': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan,

/tmp/ipykernel_2625417/3280171951.py:18: RuntimeWarning: invalid value encountered in subtract
  color_bands_over_all_inclination_angles[filters[i]+"-"+filters[i+1]] = (app_mag_final_arr[filters[i]] - app_mag_final_arr[filters[i+1]])
